So let's say you need to extract the attachment in .eml
We will be using the following example to teach you how.
https://gist.githubusercontent.com/jacobsoo/16c8abdb177c2cf9dbc3ea291cad83ef/raw/1a3c68071aa5a69a299d9fb11452a22d4c092e5f/C%25E1%25BA%25ADp%2520nh%25E1%25BA%25ADt%2520l%25E1%25BB%258Bch%2520tr%25E1%25BB%25B1c%2520l%25E1%25BB%2585%252030_4%2520v%25C3%25A0%25201_5

**SHA256** : cd075ddb7cbe9bfb9ca955be605a6f622d83bcef7eded2b495c653e86fe9b59e<br>
**ITW filename** : Cập nhật lịch trực lễ 30_4 và 1_5.eml<br>
**First Submission** : 2020-04-27 11:20:41<br><br>

Let's download the sample extract out the attachment using the script below.

In [ ]:
import zlib
import base64
import re
import sys, os, re
import requests
import binascii
from email.header import decode_header
from email.message import Message
from email.header import Header

def _log(szString):
    print(szString)

def fetch_payload(url):
    r = requests.get(url)
    return r.content.decode()

def extractAttachment(szFilePath):
    ole = ""
    hFile = open(szFilePath, 'rb')
    dataRead = hFile.read().decode()
    hFile.close()
    arrFileName = re.findall("Content-Disposition: attachment;[\W]{0,3}filename=\"(.*?)\"", dataRead, re.DOTALL|re.MULTILINE|re.IGNORECASE)
    arrFiles = re.findall("attachment;[\W]{0,3}filename=\"[^\n]{0,500}\"(.*?)--", dataRead, re.DOTALL|re.MULTILINE|re.IGNORECASE)
    szAttachment = arrFiles[0].split('\r\n\r\n')
    szAttachment = str(szAttachment[1])
    szAttachment = szAttachment.rstrip()
    szAttachment = szAttachment.lstrip()
    return arrFileName[0], szAttachment

url = 'https://gist.githubusercontent.com/jacobsoo/16c8abdb177c2cf9dbc3ea291cad83ef/raw/1a3c68071aa5a69a299d9fb11452a22d4c092e5f/C%25E1%25BA%25ADp%2520nh%25E1%25BA%25ADt%2520l%25E1%25BB%258Bch%2520tr%25E1%25BB%25B1c%2520l%25E1%25BB%2585%252030_4%2520v%25C3%25A0%25201_5'
malware_str = fetch_payload(url)
file = open("output.bin","wb")
hb = binascii.a2b_hex(malware_str)
file.write(hb)
file.close()
szFileName, g = extractAttachment("output.bin")
if "=?" in szFileName:
    tmp, idn = decode_header(szFileName)[0]
    szFileName = tmp.decode(idn)
if g:
    _log("[+] Attachment found!")
    _log("[+] Base64 Decoding now!")
    szData = base64.b64decode(g)
    if szData:
        hFile = open(szFileName, "wb")
        hFile.write(szData)
        hFile.close()
        _log("[+] Attachment extracted to {}\n".format(szFileName))
    else:
        _log("[-] Attachment seems corrupted!")
else:
    _log("[-] Attachment not found!")

[+] Attachment found!
[+] Base64 Decoding now!
[+] Attachment extracted to Lich truc le 30-4 và 1-5.xls



We have extracted the malicious attachment and found out the filename of the attachment, **Lich truc le 30-4 và 1-5.xls**<br>

Next, let's install oletools to parse the Excel file.<br>

In [ ]:
# Let's install oletools from https://github.com/decalage2/oletools
!pip install oletools

In [ ]:
from oletools.olevba import VBA_Parser, TYPE_OLE, TYPE_OpenXML, TYPE_Word2003_XML, TYPE_MHTML
print("[INFO] Parsing Lich truc le 30-4 và 1-5.xls")
vbaparser = VBA_Parser('Lich truc le 30-4 và 1-5.xls')
if vbaparser.detect_vba_macros():
    print('[INFO] VBA Macros found')
else:
    print('[INFO] No VBA Macros found')
for (filename, stream_path, vba_filename, vba_code) in vbaparser.extract_macros():
    print("-"*79)
    print("[INFO] Filename    :{}".format(filename))
    print("[INFO] OLE stream  :{}".format(stream_path))
    print("[INFO] VBA filename:{}".format(vba_filename))
    print("- "*39)
    print(vba_code)

[INFO] Parsing Lich truc le 30-4 và 1-5.xls
[INFO] VBA Macros found
-------------------------------------------------------------------------------
[INFO] Filename    :Lich truc le 30-4 và 1-5.xls
[INFO] OLE stream  :_VBA_PROJECT_CUR/VBA/ThisWorkbook
[INFO] VBA filename:ThisWorkbook.cls
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Attribute VB_Name = "ThisWorkbook"
Attribute VB_Base = "0{00020819-0000-0000-C000-000000000046}"
Attribute VB_GlobalNameSpace = False
Attribute VB_Creatable = False
Attribute VB_PredeclaredId = True
Attribute VB_Exposed = True
Attribute VB_TemplateDerived = False
Attribute VB_Customizable = True
Const pzvsvrhkmq = 2
Const rmaattwzmg = 1
Const fqkbhmquyt = 0
Public oazooxrbypbwvdxpmbny As String
Public gfvfbpbhzdtz As String
Public uxqnpprqzp As String
Public znrupomlxzfn As String
Private Sub Workbook_Open()
znrupomlxzfn = aunmljqicavx("42")
oazooxrbypbwvdxpmbny = aunmljqicavx("5574696c6d") & aunmljqicavx("616e2e657865")
gfvf

We can see that there is heavy usage of **aunmljqicavx**.<br>
This is the deobfuscate function to deobfuscate the strings.<br>
I've made a simple VB script to demostrate that
https://dotnetfiddle.net/pHxjkj<br>

But the following script is showing how i'm using oletools to simplifying things too once we find the deobfuscation method.<br>

For this particular sample, we can see that it's dropping 2 binaries.<br>
**Utilman.exe --> 80c3e22b640b47e0c41f4185f091e2c523a9ef291a75b7007303e2267b8d68c5** <BR>
**mpsvc.dll --> a369fee3b83c2d2534c48fdfac8d03a266809aaa28ace6b6002eab57cc14edd1** <BR>

In this case, **mpsvc.dll** is the malicious file.

In [ ]:
from oletools.olevba import VBA_Parser, TYPE_OLE, TYPE_OpenXML, TYPE_Word2003_XML, TYPE_MHTML
import olefile
import re, binascii

print("[INFO] Parsing Lich truc le 30-4 và 1-5.xls")
vbaparser = VBA_Parser('Lich truc le 30-4 và 1-5.xls')
for (filename, stream_path, vba_filename, vba_code) in vbaparser.extract_macros():
    print("-"*79)
    print("[INFO] VBA filename:{}".format(vba_filename))
    print("- "*39)
    lines = vba_code.split('\n')
    for line in lines:
        if "aunmljqicavx(\"" in line:
            oREString = re.compile(r"aunmljqicavx\(\"(.*?)\"\)")
            foundStrings = oREString.findall(line)
            foundStrings = ''.join(foundStrings)
            print("[INFO] Obfuscated String : {}".format(line))
            print("[INFO] Deobfuscated String : {}".format(binascii.unhexlify(foundStrings)))
vbaparser.close()

[INFO] Parsing Lich truc le 30-4 và 1-5.xls
-------------------------------------------------------------------------------
[INFO] VBA filename:ThisWorkbook.cls
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
[INFO] Obfuscated String : znrupomlxzfn = aunmljqicavx("42")
[INFO] Deobfuscated String : b'B'
[INFO] Obfuscated String : oazooxrbypbwvdxpmbny = aunmljqicavx("5574696c6d") & aunmljqicavx("616e2e657865")
[INFO] Deobfuscated String : b'Utilman.exe'
[INFO] Obfuscated String : gfvfbpbhzdtz = aunmljqicavx("6d70737663") & aunmljqicavx("2e646c6c")
[INFO] Deobfuscated String : b'mpsvc.dll'
[INFO] Obfuscated String : uxqnpprqzp = aunmljqicavx("50726f6772616d44") & aunmljqicavx("61746155706461746572") + znrupomlxzfn
[INFO] Deobfuscated String : b'ProgramDataUpdater'
[INFO] Obfuscated String : uxqnpprqzp = uxqnpprqzp + aunmljqicavx("2e6c") & aunmljqicavx("6e6b")
[INFO] Deobfuscated String : b'.lnk'
[INFO] Obfuscated String : If esqcrgszpaueiwjlalg = aunmljqicav